# Import helper functions

In [ ]:
%run ../0.shared_notebooks/0_helper_functions.ipynb

In [ ]:
# Load extra libraries
import dateutil 
import json

# Set constants

In [ ]:
CASE="DIVD-2021-00012"
SUB="set1"
IN_DIR="../IN"
OUT_DIR="../NORMALIZED"
ERR_DIR="../ERR"

In [ ]:
!ls $IN_DIR
!ls $ERR_DIR

## Defaults


In [ ]:
defaults = [
  None,  # ts_found (Timestamp when the data was "found")
  None,  # ts_leaked (Timestamp when the data was stolen/leaked)
  0,     # has_name (0/1 if the record has a name)
  0,     # has_dob (0/1 if the record has a date of birth)
  0,     # has_addr (0/1 if the record has a address)
  0,     # has_phone (0/1 if the record has a )
  0,     # has_cc (0/1 if the record has creditcard data)
  0,     # has bankacc (0/1 if the record has a bank account)
  0,     # has_ssn (0/1 if the record has a ssn)
  1,     # has ip (0/1 if the record has an ip address)
  0,     # extra_data (json object with extra data)
]
ts_from_file = False

## Read in files, guess column and type

In [ ]:
files=sorted(glob(f"{IN_DIR}/*.json"))

In [ ]:
files

## Record format for normalized files
* username
* passwd
* url (Url that the credentials belong to)
* ts_found (Timestamp when the data was "found")
* ts_leaked (Timestamp when the data was stolen/leaked)
* has_name (0/1 if the record has a name)
* has_dob (0/1 if the record has a date of birth)
* has_addr (0/1 if the record has a address)
* has_phone (0/1 if the record has a )
* has_cc (0/1 if the record has creditcard data)
* has bankacc (0/1 if the record has a bank account)
* has_ssn (0/1 if the record has a ssn)
* has ip (0/1 if the record has an ip address)
* extra_data (json object with extra data)

In [ ]:
import datetime

count=0
for file in files:
    filename=os.path.basename(file)
    timestamp=os.path.getctime(file)
    record = {}
    records = []
    with open(file, 'rb') as fh:
        for line in fh:
            line=line.decode('utf-8')
            obj=json.loads(line)
            record = {
                'username'    : "",
                'passwd'      : "",
                "url"         : "",
                "ts_found"    : "2021-05-21",
                "ts_leaked"   : "",
                "has_name"    : defaults[2],
                "has_dob"     : defaults[3],
                "has_addr"    : defaults[4],
                "has_phone"   : defaults[5],
                "has_cc"      : defaults[6],
                "has_bankacc" : defaults[7],
                "has_ssn"     : defaults[8],
                "has_ip"      : defaults[9],
                "extra_data"  : {},
            }
            if "username" in obj["_source"]:
                record['username']              = obj["_source"]["username"]
            if "password" in obj["_source"]:
                record["passwd"]                = obj["_source"]["password"]
            if "url" in obj["_source"] :
                record["url"]                   = obj["_source"]["url"]
            record["ts_leaked"]             = text2sqlitedate(obj["_source"]["basic_info"]["collection_time"])
            record["extra_data"]["browser"] = obj["_source"]["browser"]
            record["extra_data"]["index"]   = obj["_index"]
            if "ip" in obj["_source"]["basic_info"]:
                record["extra_data"]["ip"]      = obj["_source"]["basic_info"]["ip"]
            else:
                record["has_ip"] = 0
            record["extra_data"]["uid"]     = obj["_source"]["basic_info"]["uid"]
            record["extra_data"]["bot_id"]  = obj["_source"]["basic_info"]["bot_id"]
            record["extra_data"] = json.dumps(record["extra_data"])
            records.append(record)
            count=count+1
            if count % 1_000 == 0 :
                print(f"{count:,}", end="\r")
print()
df = pd.DataFrame(records)    

In [ ]:
df.head()

In [ ]:
df.to_csv(f"{OUT_DIR}/{CASE}-{SUB}.tsv", sep="\t", index=False)